# Encontrando una política para jugar el 21 (sin repartidor)

Esta libreta utiliza dos algoritmos de programación dinámica para encontrar una política que permita jugar al 21.

Las reglas para este 21 son especiales:
- El mazo es infinito. De esta manera las 13 cartas siempre tienen las mismas probabilidades de aparecer
- No se toma en cuenta al repartidor
- Si el jugador llega a 4 cartas sin pasarse automáticamente gana

Con las reglas explicadas, definiremos una estructura para procesos de decisión markoviana.

In [1]:
struct MDP
    states
    actions
    ρ
    reward
    final_s
end

## Funciones para hallar la política

### Valor de política

Primero implementaremos una función que nos permitirá calcular el valor para cada estado de nuestro Proceso de Decisión de Markov con una política $\pi$ y con factor de descuento $\gamma$.

Esta función calcula iterativamente el valor para cada estado hasta que los resultados convergen.

In [20]:
"""
    policy_value(mdp::MDP, pol_π::Dict, γ::Float64)

Computes the value for every state in the MDP by using pol_π and discount γ.
"""
function policy_value(mdp::MDP, pol_π::Dict, γ::Float64)
    v = Dict(s => 0.0 for s ∈ mdp.states)
    
    has_converged = false
    while !has_converged
        has_converged = true
    
        for s ∈ keys(v)
            temp = sum([mdp.ρ(s, pol_π[s], n_s) * (mdp.reward(s, pol_π[s], n_s) + γ*v[n_s]) for n_s ∈ keys(v)])
                    
            if temp != v[s]
                has_converged = false
            end
            
            v[s] = temp
        end             
    end
            
    return v
end

policy_value

### Iteración de política

Con la función anterior podemos programar ahora una función que encuentre una política óptima para un Proceso de Decisión de Markov.

In [18]:
"""
    policy_iteration(mdp::MDP, γ)

Finds an optimal policy for the MDP using discount factor γ.
"""
function policy_iteration(mdp::MDP, γ)
    pol_π = Dict(s => rand(mdp.actions) for s ∈ mdp.states)
    
    is_optimal = false
    while !is_optimal
        v = policy_value(mdp, pol_π, γ)
        
        is_optimal = true
        
        for s ∈ keys(v)
            for a ∈ mdp.actions
                temp = sum([mdp.ρ(s, a, n_s) * (mdp.reward(s, a, n_s) + γ*v[s]) for n_s ∈ keys(v)])
                
                if temp < v[s]
                    is_optimal = false
                    pol_π[s] = a
                end
            end
        end
    end
    
    return pol_π
end

policy_iteration

### Iteración de valor

Enseguida viene una implementación de un algoritmo iterativo para encontrar una buena política, todo en un único bloque de código.

In [4]:
"""
    iter_value(mdp::MDP, γ::Float64)

Iteratively computes the value function of a Markov Decision Process using
discount rate γ and then returns the optimal policy π associated with it.
"""
function iter_value(mdp::MDP, γ::Float64)
    v = Dict(s => 0.0 for s ∈ mdp.states)
    v_p = Dict(s => 0.0 for s ∈ mdp.states)
    
    has_converged = false
    
    while !has_converged
        for s ∈ keys(v)
            if s ∈ mdp.final_s
                v_p[s] = mdp.reward(s, nothing, s)
            else
                v_p[s] = maximum([sum([mdp.ρ(s, a, n_s) * (mdp.reward(s, a, n_s) + γ * v[n_s])
                                            for n_s ∈ mdp.states])
                                        for a ∈ mdp.actions])
            end

            has_converged = true
                            
            for s ∈ keys(v)
                if v_p[s] > v[s]
                    v[s] = v_p[s]
                    has_converged = false
                end
            end
            
            if has_converged
                break
            end
        end
    end
    
    pol_π = Dict(s => mdp.actions[1] for s ∈ mdp.states)
    
    for s ∈ keys(v)
        actions_value = Dict(a => sum([mdp.ρ(s, a, n_s) * v[n_s] for n_s ∈ mdp.states])
                            for a ∈ mdp.actions)
                                
        pol_π[s] = findmax(actions_value)[2]
    end
    
    return pol_π
end

iter_value

## Definiendo el proceso de decisión de Markov

Una vez que tenemos las funciones que nos permitirán encontrar una política óptima para el 21, ahora ocupamos definir un modelo que se ajuste a estos.
En las siguientes celdas viene el código necesario para definir un proceso de decisión de Markov que represente el juego y que sea compatible con ``policy_value``, ``policy_iteration``, ``iter_value``.

### Estados del 21

Primero definiremos los estados del juego como un arreglo de dos de números enteros.

$$(S, C)$$

Donde $S$ es la suma total de la cantidad $C$ de cartas que tiene el jugador.

In [5]:
states = [[2, i] for i in 2:26]

for i in 3:39
    push!(states, [3, i])
end

for i in 4:52
    push!(states, [4, i])
end

states

111-element Array{Array{Int64,1},1}:
 [2, 2] 
 [2, 3] 
 [2, 4] 
 [2, 5] 
 [2, 6] 
 [2, 7] 
 [2, 8] 
 [2, 9] 
 [2, 10]
 [2, 11]
 [2, 12]
 [2, 13]
 [2, 14]
 ⋮      
 [4, 41]
 [4, 42]
 [4, 43]
 [4, 44]
 [4, 45]
 [4, 46]
 [4, 47]
 [4, 48]
 [4, 49]
 [4, 50]
 [4, 51]
 [4, 52]

Podemos revisar cuantos estados fueron generados con la siguiente celda.

In [6]:
println("Cantidad de estados:", size(states))

Cantidad de estados:(111,)


### Estados finales

Una vez que tenemos todos los estados del 21, agregamos todos los estados finales a un único lugar.

In [7]:
final_states = []
for s ∈ states
    if s[1] == 4 || s[2] >= 21
        push!(final_states, s)
    end
end

final_states

74-element Array{Any,1}:
 [2, 21]
 [2, 22]
 [2, 23]
 [2, 24]
 [2, 25]
 [2, 26]
 [3, 21]
 [3, 22]
 [3, 23]
 [3, 24]
 [3, 25]
 [3, 26]
 [3, 27]
 ⋮      
 [4, 41]
 [4, 42]
 [4, 43]
 [4, 44]
 [4, 45]
 [4, 46]
 [4, 47]
 [4, 48]
 [4, 49]
 [4, 50]
 [4, 51]
 [4, 52]

### Acciones

Ahora las acciones. Esta parte es fácil, solo hay dos posibles acciones en todo momento. Esta libreta usa el vocabulario usado en los casinos.

In [8]:
actions = ["hit", "stand"]

2-element Array{String,1}:
 "hit"  
 "stand"

### Probabilidad de transición

Seguimos con la declaración de la función que calcula la probabilidad de transición entre estados.

In [9]:
function ρ(s, a, n_s)
    if a == "stand"
        if s == n_s
            return 1
        else
            return 0
        end
    else
        diff_score = n_s[2] - s[2]
        
        if n_s[1] == s[1] + 1 && diff_score >= 1 && diff_score <= 13 
            return 1/13
        end
    end
    
    return 0
end

ρ (generic function with 1 method)

### Recompensa

Completamos el modelo calculando la recompensa para cada estado. Aquí no hay ganancia ni pérdida a menos que el juego se acabe.

In [10]:
function reward(s, a, n_s)
    if (n_s[1] == 4 && n_s[2] <= 21) || n_s[2] == 21
        return 1
    elseif n_s[2] > 21
        return -1
    else
        return 0
    end
end

reward (generic function with 1 method)

## Resolviendo el problema

Como tenemos todos los preparativos listos, podemos crear un nuevo proceso de decisión de Markov que modele este 21.

In [11]:
twenty_one = MDP(states, actions, ρ, reward, final_states)

MDP(Array{Int64,1}[[2, 2], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7], [2, 8], [2, 9], [2, 10], [2, 11]  …  [4, 43], [4, 44], [4, 45], [4, 46], [4, 47], [4, 48], [4, 49], [4, 50], [4, 51], [4, 52]], ["hit", "stand"], ρ, reward, Any[[2, 21], [2, 22], [2, 23], [2, 24], [2, 25], [2, 26], [3, 21], [3, 22], [3, 23], [3, 24]  …  [4, 43], [4, 44], [4, 45], [4, 46], [4, 47], [4, 48], [4, 49], [4, 50], [4, 51], [4, 52]])

### Iteración de política

Y con esto, ahora podemos probar si nuestros algoritmos realmente cumplen su tarea. Primero probamos el algoritmo de iteración de políticas.

In [21]:
γ = 0.9
pol_π_1 = policy_iteration(twenty_one, γ)

Dict{Array{Int64,1},String} with 111 entries:
  [3, 3]  => "stand"
  [4, 4]  => "hit"
  [4, 16] => "hit"
  [3, 25] => "stand"
  [3, 31] => "stand"
  [3, 29] => "stand"
  [3, 4]  => "stand"
  [4, 8]  => "hit"
  [4, 43] => "stand"
  [4, 52] => "stand"
  [2, 2]  => "stand"
  [4, 19] => "hit"
  [4, 51] => "stand"
  [3, 17] => "hit"
  [4, 48] => "stand"
  [3, 27] => "stand"
  [2, 14] => "hit"
  [4, 22] => "stand"
  [4, 36] => "stand"
  [2, 17] => "hit"
  [3, 33] => "stand"
  [4, 27] => "stand"
  [2, 24] => "stand"
  [3, 26] => "stand"
  [4, 38] => "stand"
  ⋮       => ⋮

Ahora revisamos el valor para la política $\pi_1$ y los contenidos de la misma.

In [22]:
policy_value(twenty_one, pol_π_1, γ)

Dict{Array{Int64,1},Float64} with 111 entries:
  [3, 3]  => 0.0
  [4, 4]  => 0.0
  [4, 16] => 0.0
  [3, 25] => -10.0
  [3, 31] => -10.0
  [3, 29] => -10.0
  [3, 4]  => 0.0
  [4, 8]  => 0.0
  [4, 43] => -10.0
  [4, 52] => -10.0
  [2, 2]  => 0.0
  [4, 19] => 0.0
  [4, 51] => -10.0
  [3, 17] => -6.61538
  [4, 48] => -10.0
  [3, 27] => -10.0
  [2, 14] => -8.15444
  [4, 22] => -10.0
  [4, 36] => -10.0
  [2, 17] => -9.26391
  [3, 33] => -10.0
  [4, 27] => -10.0
  [2, 24] => -10.0
  [3, 26] => -10.0
  [4, 38] => -10.0
  ⋮       => ⋮

In [ ]:
for s in states
    println(s, ": ", pol_π_1[s])
end

### Iteración de valor

Por último, probamos el algoritmo de iteración de valor y examinamos el valor para la política $\pi_2$.

In [15]:
γ = 0.9
pol_π_2 = iter_value(twenty_one, γ)

Dict{Array{Int64,1},String} with 111 entries:
  [3, 3]  => "stand"
  [4, 4]  => "stand"
  [4, 16] => "stand"
  [3, 25] => "stand"
  [3, 31] => "stand"
  [3, 29] => "stand"
  [3, 4]  => "hit"
  [4, 8]  => "stand"
  [4, 43] => "stand"
  [4, 52] => "stand"
  [2, 2]  => "hit"
  [4, 19] => "stand"
  [4, 51] => "stand"
  [3, 17] => "stand"
  [4, 48] => "stand"
  [3, 27] => "stand"
  [2, 14] => "stand"
  [4, 22] => "stand"
  [4, 36] => "stand"
  [2, 17] => "stand"
  [3, 33] => "stand"
  [4, 27] => "stand"
  [2, 24] => "stand"
  [3, 26] => "stand"
  [4, 38] => "stand"
  ⋮       => ⋮

In [16]:
policy_value(twenty_one, pol_π_2, γ)

Dict{Array{Int64,1},Float64} with 111 entries:
  [3, 3]  => 0.0
  [4, 4]  => 1.0
  [4, 16] => 1.0
  [3, 25] => -1.0
  [3, 31] => -1.0
  [3, 29] => -1.0
  [3, 4]  => 1.9
  [4, 8]  => 1.0
  [4, 43] => -1.0
  [4, 52] => -1.0
  [2, 2]  => 1.01183
  [4, 19] => 1.0
  [4, 51] => -1.0
  [3, 17] => 0.0
  [4, 48] => -1.0
  [3, 27] => -1.0
  [2, 14] => 0.0
  [4, 22] => -1.0
  [4, 36] => -1.0
  [2, 17] => 0.0
  [3, 33] => -1.0
  [4, 27] => -1.0
  [2, 24] => -1.0
  [3, 26] => -1.0
  [4, 38] => -1.0
  ⋮       => ⋮

In [ ]:
for s in states
    println(s, ": ", pol_π_2[s])
end